In [1]:
import pandas as pd
# import functions_barcelona as fb
import datetime
# !pip install holidays_es
try:
    from holidays_es import Province
    festes=True
except:
    festes=False
    
# import numpy as np
# import os

# from sklearn.metrics import recall_score, precision_score, accuracy_score, roc_auc_score
# from sklearn.model_selection import KFold, cross_val_score, cross_validate, StratifiedKFold
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import LinearSVC
# from sklearn.preprocessing import MinMaxScaler, StandardScaler,Normalizer
# from xgboost import XGBClassifier

# # from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
# # from imblearn.under_sampling import RandomUnderSampler, NearMiss
# from sklearn.dummy import DummyClassifier
# import imblearn.under_sampling as under
# import imblearn.over_sampling as over
# import imblearn.combine as over_under
# from imblearn.pipeline import Pipeline, make_pipeline

In [2]:
accidents=pd.read_csv('./data/accidents_weather.csv')#,date_format='%Y/%m/%d %H:%M:%S%z')
accidents['datetime']=pd.to_datetime(accidents.datetime, utc=True)
#fixing street names
accidents['street_name']=accidents.street_name.str.strip()
#casting some columns to int
for col in ['year','day','hour']:
    accidents[col]=accidents[col].astype(int)
accidents.head()

,num_incident,district_code,district,neighborhood,street_name,weekday,year,month,day,hour,...,temp_avg,temp_max,temp_min,relative_humidity,precipitation,windspeed,wind_direction,max_windspeed,pressure,solar_radiation
0,2023S000001,3,Sants-Montjuïc,Sants,Badal / Càceres,Sunday,2023,January,1,10,...,11.9,12.2,11.4,94.0,0.0,7.2,227.0,11.5,1027.1,315.0
1,2023S000002,7,Horta-Guinardó,el Baix Guinardó,Encarnació,Sunday,2023,January,1,3,...,14.2,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0
2,2023S000003,1,Ciutat Vella,el Raval,Carme,Sunday,2023,January,1,3,...,14.2,14.7,13.5,58.0,0.0,8.3,257.0,13.7,1025.4,0.0
3,2023S000005,6,Gràcia,la Vila de Gràcia,Riera de Sant Miquel / Sèneca,Sunday,2023,January,1,17,...,12.8,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0
4,2023S000006,6,Gràcia,la Vila de Gràcia,Gran de Gràcia / Gràcia,Sunday,2023,January,1,17,...,12.8,13.0,12.6,53.0,0.0,4.3,229.0,10.4,1025.1,0.0


<a id='start'></a>
# Feature Engineering
 1. [**Binning streets and neighborhoods and creating a count of accidents.**](#binning) We should be careful with this one. I created a function that does the count with the train and applies it to the test. I have assumed that 2022 and 2023 are the test. I have done the count (street_count) and made 5 bins (street_bins). The same applies to neighborhoods.
 2. [**Creating a holiday column.**](#holidays) Assign 1 if holiday, 0 if previoous to holiday and -1 if non a holiday
 3. [**Identifying crossroads.**](#crossroads) It might be interesting evaluate if the fact of the accident taking place in a cross road is relevant.

<a id='binning'></a>
## 1. **Binning streets and neighborhoods and creating a count of accidents**.

   
[Back](#start)

In [3]:
def binning_fields(df,field, last_year_train,num_bins=5):
    """binning the streets for train and adding it
    without leakage to test"""
    train=df[df.year<=last_year_train]
    df[field +'_bins']=pd.qcut(
    df[field].map(
        train[field].value_counts().to_dict()),
                                  num_bins,
                                  labels=False)
    df[field+'_count']=df[field].map(train[field].value_counts().to_dict())
    df[field+'_count'].fillna(0,inplace=True)
    df[field +'_bins'].fillna(0,inplace=True)
    return df
accidents=binning_fields(accidents,'street_name',2021)

In [4]:
accidents.groupby('street_name').street_name_count.mean().sort_values(ascending=False).iloc[:10]

street_name
Corts Catalanes        4893.0
Diagonal               4101.0
Aragó                  2509.0
Litoral (Llobregat)    1872.0
Meridiana              1828.0
Dalt (Besòs)           1678.0
Litoral (Besòs)        1470.0
València               1457.0
Dalt (Llobregat)       1327.0
Gràcia                 1251.0
Name: street_name_count, dtype: float64

In [5]:
accidents.groupby('street_name_bins')['street_name_count'].mean()

street_name_bins
0.0       4.901608
1.0      41.970191
2.0     202.893561
3.0     659.870613
4.0    2869.975588
Name: street_name_count, dtype: float64

In [6]:
accidents=binning_fields(accidents,'neighborhood',2021,num_bins=3)

In [7]:
accidents.groupby('neighborhood').neighborhood_count.mean().sort_values(ascending=False).iloc[:5]

neighborhood
la Dreta de l'Eixample             12351.0
l'Antiga Esquerra de l'Eixample     6071.0
Sant Gervasi - Galvany              4631.0
la Nova Esquerra de l'Eixample      4275.0
la Sagrada Família                  3827.0
Name: neighborhood_count, dtype: float64

In [8]:
accidents.groupby('neighborhood_bins')['neighborhood_count'].mean()

neighborhood_bins
0     994.460620
1    2068.520171
2    7371.466992
Name: neighborhood_count, dtype: float64

<a id='holidays'></a>
## 2. **Creating a holiday column.**

   
[Back](#start)

In [11]:
from holidays_es import Province, Scope, HolidaySpain

#Gathering all holidays either Spanish, Catalan or local from 2010 to 2024
holidays_bcn=[]
for year in range(2010,2024):
    holidays=HolidaySpain(province=Province.BARCELONA,year=year)
    for holiday in holidays.national:
        holidays_bcn.append(holiday.date)
        
    for holiday in holidays.regional:
        holidays_bcn.append(holiday.date)
    for holiday in holidays.local:
        holidays_bcn.append(holiday.date)
#defining a function to determine if a given date (TimeStamp) is a holiday, holiday eve or none.
def adding_holiday(date_time):
    date_time=datetime.datetime.date(date_time)
    if date_time in holidays_bcn:
        return 1
    elif date_time +datetime.timedelta(days=1) in holidays_bcn:
        return 0
    else:
        return -1
#generating the new column

accidents['is_holiday']=accidents['datetime'].apply(adding_holiday).unique()

<a id='crossroads'></a>
## 3. **Identifying accidents that occur in a crossing.**

   
[Back](#start)

In [37]:
#2. Identifying accidents that occur in a crossing
accidents['crossing_street']=[1 if '/' in x else 0 for x in accidents.street_name]

In [38]:
##Saving the file with new features
accidents.to_csv('./data/accidents_weather_eng_2023.csv',index=False)